In [4]:
import matplotlib.pyplot as plt
import numpy as np
import math

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
import scipy.stats as stats
from sklearn.preprocessing import MinMaxScaler

def getAjustedR2(actual, predicted) :
    return 1 - (1 - r2_score(actual, predicted) ) * (len(predicted) - 1) / (len(predicted) - actual.shape[0] - 1); 


bx, by = datasets.fetch_california_housing(return_X_y=True)

repeats = 30
mses = []
maes = []
mapes = []
r2s = []
ar2s = []

msesSVM = []

scaler = MinMaxScaler()

for i in range(0, repeats):
    bx_train, bx_test, by_train, by_test = train_test_split(bx, by, test_size = 0.25)

    #DT
    dtm = DecisionTreeRegressor(max_depth=3)
    dtm.fit(bx_train, by_train)

    by_pred = dtm.predict(bx_test)
    mses.append(mean_squared_error(by_test, by_pred))
    maes.append(mean_absolute_error(by_test, by_pred))
    mapes.append(mean_absolute_percentage_error(by_test, by_pred))
    r2s.append(r2_score(by_test, by_pred))
    ar2s.append(getAjustedR2(by_test, by_pred))

    #SVM
    fit_scalar = scaler.fit(bx_train)
    bx_train_scaled = fit_scalar.transform(bx_train)

    svmm = svm.SVR(kernel="rbf")
    svmm.fit(bx_train_scaled, by_train)

    bx_test_scaled = fit_scalar.transform(bx_test)
    by_predSVM = svmm.predict(bx_test_scaled)

    msesSVM.append(mean_squared_error(by_test, by_predSVM))


print("MSE:", np.mean(mses))
print("RMSE:", math.sqrt(np.mean(mses)))
print("MAE:", np.mean(maes))
print("MAPE:", np.mean(mapes))
print("R^2:", np.mean(r2s))
print("Adjusted R^2:", np.mean(ar2s))
print(stats.wilcoxon(mses, msesSVM))

MSE: 0.6298681292084481
RMSE: 0.793642318181464
MAE: 0.5951135661847712
MAPE: 0.3756275522221932
R^2: 0.5256285750579173
Adjusted R^2: 2448.2821812762045
WilcoxonResult(statistic=0.0, pvalue=1.862645149230957e-09)
